In [5]:
import json
import pandas as pd
import numpy as np
import math
import datetime

In [ ]:
import requests
import hashlib
# 构造 URL
# 第一个url为指数信息 第二个为基金信息
# base_url = 'http://mallapi.fof99.com/index/price'
base_url = 'http://mallapi.fof99.com/price'
# 定义参数
# SQH719
params = {
    #填写你的app_id
    "app_id": "",
    #想要查询的基金代码
    'reg_code': 'SS9013',
    'order_by': 'price_date',
    # 时间排序 0=倒序，1=升序	
    'order': '1',
    # 开始日期。 不传默认全部 Format = YYYY-MM-DD	
    'start_date': '2022-01-01'
}

In [ ]:
# 定义秘钥
AppSecret= "YOUR-APP_SECRET_CODE"
# 对参数按照 key 升序排序，并拼接成 key=value 形式的字符串
sorted_params = sorted(params.items(), key=lambda x: x[0])
query_string = '&'.join([f'{k}={v}' for k, v in sorted_params])
# 在拼接上秘钥
query_string_with_key = query_string + AppSecret
# 计算字符串的 MD5 哈希
hash_object = hashlib.md5(query_string_with_key.encode())
sign = hash_object.hexdigest()
# 将签名添加到参数中
params['sign'] = sign

url = f'{base_url}?{"&".join([f"{k}={v}" for k, v in params.items()])}'
response = requests.get(url)

{'error_code': 0,

 'msg': '',

 'data': [{'nav': 1.821,

   'cumulative_nav_withdrawal': 1.821,

   'cumulative_nav': 1.821,

   'price_change': -0.0043739748,

   'price_date': '2023-09-22'},

  {'nav': 1.829,

   'cumulative_nav_withdrawal': 1.829,

   'cumulative_nav': 1.829,

   'price_change': 0.0155469184,

   'price_date': '2023-09-15'},

  {'nav': 1.801,

   'cumulative_nav_withdrawal': 1.801,

   'cumulative_nav': 1.801,

   'price_change': -0.0120680197,

   'price_date': '2023-09-08'}]
   
}

   格式如上

In [3]:
a=[
    {'参数名':'code','描述':'0=成功，大于0请求出错','类型':'string'},
    {'参数名':'data.nav','描述':'当前净值','类型':'string'},
    {'参数名':'data.cumulative_nav_withdrawal','描述':'累计净值(分红不投)','类型':'string'},
    {'参数名':'data.cumulative_nav','描述':'累计净值(分红再投)','类型':'string'},
    {'参数名':'data.price_change','描述':'净值变化','类型':'string'},
    {'参数名':'data.price_date','描述':'净值日期','类型':'string'}
]

In [7]:
a = pd.DataFrame(a)
a

,参数名,描述,类型
0,code,0=成功，大于0请求出错,string
1,data.nav,当前净值,string
2,data.cumulative_nav_withdrawal,累计净值(分红不投),string
3,data.cumulative_nav,累计净值(分红再投),string
4,data.price_change,净值变化,string
5,data.price_date,净值日期,string


In [ ]:
# 读取json格式的基金净值信息
dict = json.loads(response.text)
dict

In [9]:
# 这里我们选择nav作为净值进行后续计算
df_nav = pd.DataFrame()
for item in dict['data']:
    temp = pd.DataFrame({'nav': item['nav'],\
                            'price_date': item['price_date'],\
                                'price_change': item['price_change']},index=[0])
    df_nav = pd.concat([df_nav,temp],ignore_index=True)
df = df_nav
name = 'nav'

In [ ]:
# 初试日期净值归一化
newValues = []
firstValue = df.iloc[-1]
if name == 'cum_nav_withdraw':
    len_value = len(str(firstValue.cum_nav_withdraw).split('.')[0]) - 1
    bit = 10 ** len_value
    scale = (firstValue.cum_nav_withdraw / bit) - 1
else:
    len_value = len(str(firstValue.nav).split('.')[0]) - 1
    bit = 10 ** len_value
    scale = (firstValue.nav / bit) - 1

df['normalize'] = None

for i in df.index:
    value = df.at[i, name]
    normalized_value = (value / bit) - scale
    df.at[i, 'normalize'] = normalized_value

In [ ]:
# 日期转换为datetime格式
df['price_date'] = pd.to_datetime(df['price_date'])

In [ ]:
from empyrical import max_drawdown as md,sharpe_ratio as sr,calmar_ratio as cr,annual_return as ar
# 加载empyrical来计算回撤 夏普 卡玛 值